In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [2]:
load_dotenv(override=True)

True

In [3]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [6]:
documents = recur_split.split_documents(file)

# Embbiding (OpenAI)

In [7]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [8]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# LLM (OpenAI)

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", base_url="", api_key=api_key, temperature=0)

# Chain

In [10]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [11]:
question = "O que é Identificar oportunidades?"

# Answer

In [12]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?', 'result': 'Não sei.'}

# Prompt

In [23]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Você é um assistente de extração de dados educacionais.  
Sua tarefa é localizar **exatamente** os Códigos e Descrições **que já estão presentes no contexto abaixo**.  
**Você NÃO deve criar, reescrever, resumir ou modificar nenhuma informação.**

{context}

REGRAS IMPORTANTES:
- Você **NÃO** pode inventar nenhum conteúdo que não esteja literalmente no contexto acima.
- Você deve retornar **apenas os códigos e descrições exatos e completos** que aparecem no contexto.
- Sempre traga o Codigo e Descricao mais proximos dos dados fornecidos"
- **Não corte partes das descrições.**
- **Não relacione o conteúdo com códigos errados.**

FORMATO OBRIGATÓRIO DE SAÍDA (copiar literalmente do contexto):
Código: <código exato>  
Descrição: <descrição completa>

---

Pergunta do usuário:  
{question}
"""
)


# Chain

In [24]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [29]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <experimentos aleatórios> e o **Objetivo da Aula** é <reconhecer>."

# Answer

In [30]:
chat_chain.invoke({"query": question})

{'query': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <experimentos aleatórios> e o **Objetivo da Aula** é <reconhecer>.',
 'result': 'Código: EF09MA19  \nDescrição: Reconhecer, em experimentos aleatórios, eventos independentes e dependentes e calcular a probabilidade de sua ocorrência, nos dois casos.',
 'source_documents': [Document(metadata={'source': '../pdf/matematica.pdf', 'creationdate': 'D:20250429004401', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'page_label': '1', 'page': 0, 'total_pages': 3, 'creator': 'PyPDF'}, page_content='Matemática - 9º Ano\nCódigo: EF09MA01\nDescrição: Reconhecer que, uma vez fixada uma unidade de comprimento, existem segmentos de reta cujo\ncomprimento não é expresso por número racional, como as medidas de diagonais de um polígono e alturas\nde um triângulo.\nCódigo: EF09MA02\nDescrição: Reconhecer um número irracional como um número real cuja representação decimal é infinita e\nnão periódica, e estimar a